<a href="https://colab.research.google.com/github/impanaj07/bidirectional-transformer-model/blob/main/fake_news_detection_using_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets scikit-learn pandas numpy torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset


In [3]:
from google.colab import files

# Upload your dataset (e.g., "Fake.csv" and "True.csv" from Kaggle)
uploaded = files.upload()

# Load real and fake news
fake_df = pd.read_csv("Fake.csv")
real_df = pd.read_csv("True.csv")

# Labeling: fake = 0, real = 1
fake_df["label"] = 0
real_df["label"] = 1

# Merge and shuffle
data = pd.concat([fake_df, real_df])
data = data.sample(frac=1).reset_index(drop=True)

# Use only title for simplicity
data = data[['title', 'label']]
data.dropna(inplace=True)


Saving True.csv to True.csv
Saving Fake.csv to Fake.csv


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    data["title"], data["label"], test_size=0.2, random_state=42
)


In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class NewsDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(
            texts.tolist(), truncation=True, padding=True, max_length=64
        )
        self.labels = labels.tolist()

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(X_train, y_train)
test_dataset = NewsDataset(X_test, y_test)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
!pip install -U transformers


In [8]:
import transformers
print(transformers.__version__)


4.51.3


In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
)


In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        p.label_ids, preds, average="binary"
    )
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


Step,Training Loss
500,0.018300
1000,0.052300
1500,0.048100
2000,0.056400
2500,0.058000
3000,0.077000
3500,0.029900
4000,0.038500
4500,0.031900
5000,0.025600


TrainOutput(global_step=13470, training_loss=0.02765980864243766, metrics={'train_runtime': 1872.3338, 'train_samples_per_second': 57.551, 'train_steps_per_second': 7.194, 'total_flos': 3488535010807560.0, 'train_loss': 0.02765980864243766, 'epoch': 3.0})

In [31]:
eval_result = trainer.evaluate()
print(eval_result)

# Get predictions for final report
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)

print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))


{'eval_loss': 0.09018366783857346, 'eval_accuracy': 0.9870824053452116, 'eval_f1': 0.9865926953305594, 'eval_precision': 0.987505784359093, 'eval_recall': 0.9856812933025404, 'eval_runtime': 32.1113, 'eval_samples_per_second': 279.653, 'eval_steps_per_second': 34.972, 'epoch': 3.0}

Classification Report:
               precision    recall  f1-score   support

           0     0.9867    0.9884    0.9875      4650
           1     0.9875    0.9857    0.9866      4330

    accuracy                         0.9871      8980
   macro avg     0.9871    0.9870    0.9871      8980
weighted avg     0.9871    0.9871    0.9871      8980



In [35]:
import torch

# Example headline
new_headline = ["Tomato juice has been declared a cure for COVID-19."]

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
model.to(device)

# Tokenize the new headline
inputs = tokenizer(new_headline, padding=True, truncation=True, return_tensors="pt").to(device)

# Predict using the trained model
with torch.no_grad():  # optional, prevents gradient calculation
    outputs = model(**inputs)

# Get the predicted class (fake or real)
prediction = torch.argmax(outputs.logits, dim=1)

# Print prediction
print(f"Prediction: {'Real' if prediction.item() == 1 else 'Fake'}")


Prediction: Fake
